# Integrantes sala 7
- Guillermo Jair Montiel Juárez
- Juan Nicolás Pinilla Morales
- Melesio Reyes Perez
- Felipe Paez Gonzales
- Daniel Caballero

In [1]:
!python -m spacy download es_core_news_sm
!pip install fasttext
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.es.300.bin.gz
!gunzip cc.es.300.bin.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 27.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313506 sha256=a168a6012d7229dcce6190e77f946519e83322bccf99dd7a5031d4e3083619a6
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a51

In [2]:
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from collections import Counter

In [3]:
# Cargar el modelo de spaCy
nlp = spacy.load("es_core_news_sm")

In [12]:
# 1. Seleccionar dos textos cortos
texto_a = "La inteligencia artificial está revolucionando la medicina. Los algoritmos de IA pueden diagnosticar enfermedades con alta precisión. Esto transformará el futuro de la salud y el bienestar humano."
texto_b = "El aprendizaje automático es una rama de la inteligencia artificial. Se utiliza para crear modelos predictivos y mejorar la toma de decisiones. Su impacto en la industria tecnológica es innegable."

In [13]:
# Creación del objeto doc (Preprocesa cada texto)
doc_a = nlp(texto_a.lower())
doc_b = nlp(texto_b.lower())

In [14]:
# Preprocesamiento
def preprocesar_doc(doc):
    """
    - Elimina signos de puntuación y stopwords
    - Lematiza cada palabra
    """
    return [
        token.lemma_ for token in doc
        if not token.is_punct and not token.is_stop
    ]

tokens_a = preprocesar_doc(doc_a)
tokens_b = preprocesar_doc(doc_b)

print("Tokens limpios A:", tokens_a)
print("Tokens limpios B:", tokens_b)

Tokens limpios A: ['inteligencia', 'artificial', 'revolucionar', 'medicina', 'algoritmo', 'ia', 'diagnosticar', 'enfermedad', 'alto', 'precisión', 'transformar', 'futuro', 'salud', 'bienestar', 'humano']
Tokens limpios B: ['aprendizaje', 'automático', 'rama', 'inteligencia', 'artificial', 'utilizar', 'crear', 'modelo', 'predictivo', 'mejorar', 'toma', 'decisión', 'impacto', 'industria', 'tecnológico', 'innegable']


In [15]:
# Análisis gramatical (POS tagging)
def pos_frecuentes_doc(doc):
    """
    - Obtiene la categoría gramatical (POS) de cada token
    - Cuenta las etiquetas más frecuentes
    """
    etiquetas = [token.pos_ for token in doc if not token.is_punct]
    return Counter(etiquetas).most_common()

print("\nEtiquetas POS más frecuentes en A:", pos_frecuentes_doc(doc_a))
print("Etiquetas POS más frecuentes en B:", pos_frecuentes_doc(doc_b))



Etiquetas POS más frecuentes en A: [('NOUN', 9), ('DET', 6), ('ADJ', 3), ('VERB', 3), ('ADP', 3), ('AUX', 2), ('PRON', 1), ('CCONJ', 1)]
Etiquetas POS más frecuentes en B: [('NOUN', 8), ('DET', 6), ('ADJ', 5), ('ADP', 4), ('VERB', 3), ('AUX', 2), ('PRON', 1), ('CCONJ', 1)]


# Reflexion: ¿Qué categorías predominan en cada uno?
En ambos textos, predominan los sujetos (nouns), y esto coincide con el texto, debido a que en ambos se del sujeto como tema central de la oración (en este caso específico, sobre la inteligencia artificial).

In [16]:
# Vectorización con TF-IDF

# Creamos el corpus con ambos textos
corpus = [texto_a, texto_b]
# Usamos las stopwords de spaCy para español
vectorizer = TfidfVectorizer(stop_words=list(nlp.Defaults.stop_words))
# Calculamos la matriz TF-IDF
X = vectorizer.fit_transform(corpus)

In [17]:
# Similitud coseno
# Calculamos la similitud del coseno entre los dos textos
similitud = cosine_similarity(X[0], X[1])[0][0]
print(f"\nSimilitud coseno (TF-IDF): {similitud:.3f}")


Similitud coseno (TF-IDF): 0.070


In [18]:
# Vectorización con embeddings de spaCy
vector_a = nlp(texto_a).vector
vector_b = nlp(texto_b).vector

In [19]:
similitud_emb = cosine_similarity([vector_a], [vector_b])[0][0]
print(f"Similitud coseno (spaCy embeddings): {similitud_emb:.3f}")

Similitud coseno (spaCy embeddings): 0.817


# Interpretación de resultados

## 1. ¿Los textos se parecen?
Sí, en este caso particular, los textos son muy similares, porque hablan sobre la Inteligencia Artificial o el Aprendizaje Automático, y es tema central en ambos, tratándolo como un sujeto. Por ello es que usa muchos adjetivos y determinantes, que complementan las características de este sujeto. Además, la similitud del coseno coincide con lo previsto, pues es muy cercana al 1.

## 2. ¿Qué influye más en la similitud: el tema, el vocabulario o la estructura?
Para este método de vectorización en particular, se considera más el vocabulario a la hora hacer los embeddings. Si bien, la estructura completa es la que le da contexto a las palabras, la lemanización permite reducir las palabras a su mínimo estado base, perdiendo parte de su estructura en el proceso, para después vectorizar las palabras y tomar el promedio de los embeddings que conforman el texto.